In [1]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from scoop.msg import stone_pose_hori
from scoop.msg import stone_pose_ver
from sensor_msgs.msg import Image, CameraInfo
import message_filters
from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import random
import urx
import math3d as m3d
import logging
import matplotlib.pyplot as plt
rospy.init_node('scoop', anonymous=True) #initialize the node

In [3]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1609480331.001569]: Client test: Starting sending goals


In [4]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()
servo_pub = rospy.Publisher('servo1', UInt16, queue_size=10)

In [5]:
def go_to_home():
    
    Hong_joint0 = radians(101.23)
    Hong_joint1 = radians(-83.77)
    Hong_joint2 = radians(-130.28)
    Hong_joint3 = radians(-55.97)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-78.73)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.74)
    Hong_joint1 = radians(-76.84)
    Hong_joint2 = radians(-120.83)
    Hong_joint3 = radians(-72.33)
    Hong_joint4 = radians(89.40)
    Hong_joint5 = radians(-78.12)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

In [6]:
def stone_position_in_the_world(pose_x, pose_y, pose_z):
    if pose_z==0.0:
        return [None, None, None]
    rob.set_tcp((0, 0.0, 0, 0, 0, 0))    
    camPstone = np.array([pose_x, pose_y, pose_z, 1])
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
    tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
    worldTstone = np.matmul(rob.get_pose().get_matrix(), tcpTstone)
    worldTstone = worldTstone.squeeze().tolist()[0][0:-1]
    return [worldTstone[0]+0.01953+0.0016-0.003, worldTstone[1]+0.02148+0.0029-0.004, worldTstone[2]]

In [7]:
def hori_stone_pose_callback(msg):
    global hori_pose_Go_stone_set, hori_Go_stone_number, Go_stone_dir_sorted, yaw_pitch_normal_set
    hori_Go_stone_number = int(msg.hori_Go_stone_number)
    if hori_Go_stone_number == 0:
        hori_pose_Go_stone_set = []
        Go_stone_dir_sorted = []
        yaw_pitch_normal_set = []
    else:
        pose_x = msg.x
        pose_y = msg.y
        pose_z = msg.z
        yaw = msg.yaw
        pitch = msg.pitch
        normal = msg.normal
        dir_depth_inner = msg.dir_depth_inner
        dir_depth_outer = msg.dir_depth_outer
        dir_depth_outer_minus_inner = []
        for i in range(len(dir_depth_inner)):
            dir_depth_outer_minus_inner.append(dir_depth_outer[i]-dir_depth_inner[i])
        dir_depth_outer_minus_inner_argsort = np.argsort(-np.array(dir_depth_outer_minus_inner)).tolist()
        dir_depth_outer_minus_inner_argsort_copy = dir_depth_outer_minus_inner_argsort[:]
        for index in dir_depth_outer_minus_inner_argsort:
            if dir_depth_outer[index]-dir_depth_inner[index]<0:
                dir_depth_outer_minus_inner_argsort_copy.remove(index)
            elif dir_depth_outer[index]==0 or dir_depth_inner[index]==0:
                dir_depth_outer_minus_inner_argsort_copy.remove(index)
        dir_depth_outer_minus_inner_argsort=dir_depth_outer_minus_inner_argsort_copy[:]
        hori_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
        Go_stone_dir_sorted.append(dir_depth_outer_minus_inner_argsort)
        yaw_pitch_normal_set.append([yaw,pitch,normal])
hori_pose_topic = '/hori_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(hori_pose_topic, stone_pose_hori, hori_stone_pose_callback)

In [8]:
import json
def ver_stone_pose_callback(msg):
    global ver_pose_Go_stone_set, ver_Go_stone_number, gripper_direction_set, gripper_aperture_set
    ver_Go_stone_number = int(msg.ver_Go_stone_number)
    if ver_Go_stone_number == 0:
        ver_pose_Go_stone_set = []
        gripper_direction_set = []
        gripper_aperture_set = []
    else:
        pose_x = msg.x
        pose_y = msg.y
        pose_z = msg.z 
        box = json.loads(msg.box)
        ver_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
        box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
        #print box
        length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
        length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
        if length1 > length2:
            try:
                if abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2)<abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2):
                    temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2
                else:
                    temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2
                gripper_direction_set.append(temp_angle)
            except:
                gripper_direction_set.append(0)
            gripper_aperture_set.append(length2+0.004)
        else:
            try:
                if abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2)<abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2):
                    temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2
                else:
                    temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2
                gripper_direction_set.append(temp_angle)
            except:
                gripper_direction_set.append(0)
            gripper_aperture_set.append(length1+0.004)        
ver_pose_topic = '/ver_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(ver_pose_topic, stone_pose_ver, ver_stone_pose_callback)

In [9]:
is_send_stone_img = 0
stone_img_count = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img, stone_img_count, cv2_img, cv2_depth_img
    if is_send_stone_img == 1:
        print "dsag"
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('/home/terry/Mask_RCNN/samples/stones/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        cv2.imwrite('/home/terry/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('/home/terry/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0

In [10]:
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

0

In [19]:
def poke_grasp_execute(Go_index):
    global hori_pose_Go_stone_set, yaw_pitch_normal_set
    pose_x = hori_pose_Go_stone_set[Go_index][0]
    pose_y = hori_pose_Go_stone_set[Go_index][1]
    pose_z = hori_pose_Go_stone_set[Go_index][2]
    normal=yaw_pitch_normal_set[Go_index][2]
    ####################### change tcp pose ############################
    rob.set_tcp((0.011, 0, 0.342, 0, 0, 0)) # need change
    rospy.sleep(0.5)
    ####################################################################
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=0.021, speed=0.5, force=10, block=True)
    servo_pub.publish(80)     # need change
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))   
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    
    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.342,-0.011,0])   # need change
    ####################################################################
    tcpTee = inv(eeTtcp)

    n_normalize = normal / np.linalg.norm(normal)
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(eeTcam.get_matrix(), n_normalize)
    tcpTnormal = np.matmul(tcpTee, np.transpose(eeTnormal))
    alpha = atan2(tcpTnormal[1], tcpTnormal[0])
    if str(alpha) == 'nan':
        return False
    beta = -(pi/2+atan2(tcpTnormal[2], sqrt(tcpTnormal[0]**2+tcpTnormal[1]**2)))
    if str(beta) == 'nan':
        return False
    if abs(beta)>(35*pi/180.0):
        return False
    
    move = m3d.Transform((tcpTstone[0,0], tcpTstone[1,0], 0, 0, 0, 0))
    rob.add_pose_tool( move, acc=0.3, vel=0.8, wait=True, command="movel", threshold=None)
    rob.translate_tool((-0.002, -0.002, tcpTstone[2,0]-0.015), acc=0.3, vel=0.8)
    rob.movel_tool((0,0,0,0,0,alpha), acc=0.5, vel=0.8, wait=True)
    rob.movel_tool((0,0,0,0,beta,0), acc=0.5, vel=0.8, wait=True)
    ###########################Choose d and psi######################################
    rob.translate_tool((0.004+0.0015*int(abs(beta)>10)+0.003*int(abs(beta)>15)+0.001, 0, 0), acc=0.06, vel=0.08)
    psi = radians(20)
    rob.movel_tool((0,0,0,0,psi,0), acc=0.3, vel=0.8, wait=True)
    #################################################################################

    rob.translate_tool((0, 0, 0.0285), acc=0.01, vel=0.05, wait=False) #2.85 a003 v005  $0.285
    
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.force.z
    time0 = time.time()
    IsStack = False
    while True:
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.force.z<ini_wrench-2.5:   # need change
            IsStack = True
            rob.stopl()
            break
        if time.time()-time0>3.2:  #3.7
            IsStack = False
            break
    print 'IsStack', IsStack
    
    if IsStack == False:
        servo_pub.publish(180)
        rospy.sleep(0.5)
        Robotiq.goto(robotiq_client, pos=0.006, speed=0.5, force=10, block=False)
        rospy.sleep(.5)
        rob.translate_tool((0, 0, -0.08), acc=0.3, vel=0.8)
        go_to_end_place()
        rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
        Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
        rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)
        rob.set_tcp((0, 0, 0, 0, 0, 0))
        rospy.sleep(0.5)
    elif IsStack == True:
        current_alpha = round((0.021-1.12/48)/(0.14/48))   # need change
        angle_step = 0.2
        l0 = 0.0125
        l1 = 0.1
        l2r = 0.006
        flength_r = 0.126
        for alpha in np.arange(current_alpha,2,angle_step):
            next_alpha = alpha+0.2
            alpha = alpha*pi/180
            next_alpha = next_alpha*pi/180
            translate_distance = (np.array([l0+l1*sin(next_alpha)-l2r, -l1*cos(next_alpha)-flength_r])-np.array([l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r])).tolist()
            Robotiq.goto(robotiq_client, pos=(next_alpha*180/pi)*0.14/48.0+1.12/48.0, speed=0.4, force=10, block=False)
            rob.translate_tool((-translate_distance[0], 0, -translate_distance[1]), acc=0.1, vel=0.4, wait=True)       
        servo_pub.publish(115)
        rob.set_tcp((0.019, 0, 0.342, 0, 0, 0)) # need change!!!
        rospy.sleep(0.5)
        minus_x = -rob.get_pose().get_matrix()[0:3,0]
        minus_x_proj = [minus_x[0], minus_x[1], 0]
        minus_z = -rob.get_pose().get_matrix()[0:3,2]
        rotate_angle = -(60*pi/180-acos(np.dot(minus_x_proj, minus_z)/(np.linalg.norm(minus_x_proj)*np.linalg.norm(minus_z))))
        rob.movel_tool((0,0,0,0,rotate_angle,0), acc=0.3, vel=0.2, wait=True)
        rob.set_tcp((0, 0, 0, 0, 0, 0))
        rospy.sleep(0.5)
        current_fingerlength_int=115   #need change
        angle_step=0.2
        for angle in np.arange(2,-4,-angle_step):
            translate_dir_dis, strench_distance = pos_flength_adjust_right_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.015, 0.006, 0.126)
            old_fingerlength_int=current_fingerlength_int
            current_fingerlength_int=round(min(current_fingerlength_int+strench_distance*(180.0/0.03),180))
            servo_pub.publish(int(current_fingerlength_int))
            Robotiq.goto(robotiq_client, pos=angle*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        Robotiq.goto(robotiq_client, pos=-4*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
        rospy.sleep(0.5)
        rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)
        go_to_end_place()
        rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
        Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
        rospy.sleep(0.5)
        servo_pub.publish(180)
        Robotiq.goto(robotiq_client, pos=0.013, speed=0.5, force=10, block=True)
        rospy.sleep(1)
        servo_pub.publish(10)
        rospy.sleep(1)
        rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True) 
    return True

In [12]:
from shapely.geometry import LineString
from shapely.geometry import Point

def scoop_collision_check(Go_index):
    global hori_pose_Go_stone_set, Go_stone_dir_sorted, ori_index
    for ori_index in Go_stone_dir_sorted[Go_index]:
        hard_fingertip_position = [hori_pose_Go_stone_set[Go_index][0]+0.013*sin(ori_index*45*pi/180), hori_pose_Go_stone_set[Go_index][1]-0.013*cos(ori_index*45*pi/180)]
        gripper_orientation = ori_index*45
        hard_fingertip_position1 = [hard_fingertip_position[0]+0.01*cos(gripper_orientation*pi/180), hard_fingertip_position[1]+0.01*sin(gripper_orientation*pi/180)]
        hard_fingertip_position2 = [hard_fingertip_position[0]-0.01*cos(gripper_orientation*pi/180), hard_fingertip_position[1]-0.01*sin(gripper_orientation*pi/180)]
        if LineString([hard_fingertip_position1, hard_fingertip_position, hard_fingertip_position2]).within(Point([0.05,0.69]).buffer(0.055))==False:
            continue
        soft_fingertip_position = [hori_pose_Go_stone_set[Go_index][0]+(-0.015)*sin(ori_index*45*pi/180), hori_pose_Go_stone_set[Go_index][1]-(-0.015)*cos(ori_index*45*pi/180)]
        if LineString([[soft_fingertip_position[0]+0.01*cos(gripper_orientation*pi/180), soft_fingertip_position[1]+0.01*sin(gripper_orientation*pi/180)], soft_fingertip_position, [soft_fingertip_position[0]-0.01*cos(gripper_orientation*pi/180), soft_fingertip_position[1]-0.01*sin(gripper_orientation*pi/180)]]).within(Point([0.05,0.69]).buffer(0.055))==False:
            continue
        fingertip_extend_position = [hori_pose_Go_stone_set[Go_index][0]+0.15*sin(ori_index*45*pi/180), hori_pose_Go_stone_set[Go_index][1]-0.15*cos(ori_index*45*pi/180)]
        fingertip_extend_position1 = [fingertip_extend_position[0]+0.01*cos(gripper_orientation*pi/180), fingertip_extend_position[1]+0.01*sin(gripper_orientation*pi/180)]
        fingertip_extend_position2 = [fingertip_extend_position[0]-0.01*cos(gripper_orientation*pi/180), fingertip_extend_position[1]-0.01*sin(gripper_orientation*pi/180)]
        intersect_at_bowl = list(Point([0.05,0.69]).buffer(0.065).boundary.intersection(LineString([hard_fingertip_position, fingertip_extend_position])).coords[:][0])
        intersect_at_bowl1 = list(Point([0.05,0.69]).buffer(0.065).boundary.intersection(LineString([hard_fingertip_position1, fingertip_extend_position1])).coords[:][0])
        intersect_at_bowl2 = list(Point([0.05,0.69]).buffer(0.065).boundary.intersection(LineString([hard_fingertip_position2, fingertip_extend_position2])).coords[:][0])
        if Point(hard_fingertip_position).distance(Point(intersect_at_bowl))*sqrt(3)<0.07-(hori_pose_Go_stone_set[Go_index][2]-0.03): # need to be changed
            continue
        if Point(hard_fingertip_position1).distance(Point(intersect_at_bowl1))*sqrt(3)<0.07-(hori_pose_Go_stone_set[Go_index][2]-0.03): # need to be changed
            continue
        if Point(hard_fingertip_position2).distance(Point(intersect_at_bowl2))*sqrt(3)<0.07-(hori_pose_Go_stone_set[Go_index][2]-0.03): # need to be changed
            continue
        return False
    return True            

In [13]:
from math import *
import numpy as np
def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

# first shorten, next translate, finally close
# l0=0.0125 l1=0.1 l2l=0.006 l2r=0.015 flength_l = 0.072   theta=60    joint_value = 78.88, -112.01, -106.91, -61.26, 133.28, -15.20
def pos_flength_adjust_right_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r): # right longer
    theta=theta*pi/180
    current_flength_l = flength_r - (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)
    translate_in_g = (np.array(current_right_fingertip_pos_g)-np.array(next_right_fingertip_pos_g)).tolist()
    translate_in_w = (np.squeeze(np.dot(np.array([[sin(theta), -cos(theta)], [cos(theta), sin(theta)]]), np.array([[translate_in_g[0]], [translate_in_g[1]]])))).tolist()
    next_left_fingertip_pos_g = (np.array(next_left_fingertip_pos_g)+np.array(translate_in_w)).tolist()
    next_right_fingertip_pos_g = (np.array(next_right_fingertip_pos_g)+np.array(translate_in_w)).tolist()
    next_ought_flength_l = abs(next_right_fingertip_pos_g[1]) - (2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r)/tan(theta)
    delta_finger_distance = next_ought_flength_l - abs(next_left_fingertip_pos_g[1])
    return translate_in_w, delta_finger_distance

In [14]:
from geometry_msgs.msg import WrenchStamped
from std_msgs.msg import UInt16
import time

def scoop_execute(instance_index):
    global hori_pose_Go_stone_set, ori_index
    gripper_orientation = ori_index*45
    hard_fingertip_position = [hori_pose_Go_stone_set[instance_index][0]+0.013*sin(ori_index*45*pi/180), hori_pose_Go_stone_set[instance_index][1]-0.013*cos(ori_index*45*pi/180)]
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=5*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
    servo_pub.publish(85) # need change
    rob.movel_tool((0,0,0,0,0,-gripper_orientation*pi/180),acc=0.5, vel=2)
    tcp2fingertip=m3d.Transform()
    tcp2fingertip.pos.x = -0.012 # need change
    tcp2fingertip.pos.z = 0.327 # need change
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0
    world2tcp.pos.y+=0.003
    world2fingertip=(world2tcp*tcp2fingertip)
    rob.translate((hard_fingertip_position[0]-world2fingertip.pos.x,hard_fingertip_position[1]-world2fingertip.pos.y,0), acc=0.5, vel=2, wait=True)
    rob.set_tcp((-0.012, 0, 0.327, 0, 0, 0))  # need change
    rospy.sleep(0.5)
    rob.movel_tool((0,0,0,0,pi/6,0), acc=0.5, vel=2, wait=True)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    rob.translate((0, 0, hori_pose_Go_stone_set[instance_index][2]-world2fingertip.pos.z+0.002), acc=0.02, vel=0.05, wait=True)
    current_fingerlength_int=85   #need change
    angle_step=0.2
    for angle in np.arange(5,-4,-angle_step):
        translate_dir_dis, strench_distance = pos_flength_adjust_right_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.015, 0.006, 0.126)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(min(current_fingerlength_int+strench_distance*(180.0/0.03),180))
        servo_pub.publish(int(current_fingerlength_int))
        Robotiq.goto(robotiq_client, pos=angle*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
        rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
    Robotiq.goto(robotiq_client, pos=-4*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(0.5)
    servo_pub.publish(180)
    Robotiq.goto(robotiq_client, pos=0.013, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    servo_pub.publish(10)
    rospy.sleep(1)
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True) 

In [15]:
def find_alpha_from_distance(distance, l0, l1, l2l, l2r):
    alpha = asin((distance+l2l+l2r-2*l0)/2/l1)*180/pi
    return alpha

def direct_pick_execute(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=0.75*gripper_aperture_set[Go_index]+0.0095, speed=0.1, force=10, block=True) # need change
    servo_pub.publish(180)    
    pos_x = rob.get_pose().pos.x+0.007
    pos_y = rob.get_pose().pos.y-0.001
    rospy.sleep(0.5)
    rob.translate((ver_pose_Go_stone_set[Go_index][0]-pos_x,ver_pose_Go_stone_set[Go_index][1]-pos_y,0), acc=0.8, vel=2, wait=True)
    rob.movel_tool((0,0,0,0,0,gripper_direction_set[Go_index]),acc=0.8, vel=2)
    rob.translate_tool((-0.0045,0,0),acc=0.1, vel=0.2)   # need change
    #rob.translate((0,0,-0.025), acc=0.1, vel=0.4, wait=True)
    rob.translate((0,0,max((ver_pose_Go_stone_set[Go_index][2]-(rob.get_pose().pos.z-0.32210))*0.7,-0.065)), acc=0.1, vel=0.1, wait=True)
    rob.translate((0,0,(ver_pose_Go_stone_set[Go_index][2]-(rob.get_pose().pos.z-0.32210))*0.3-0.003), acc=0.001, vel=0.01, wait=True)
    Robotiq.goto(robotiq_client, pos=-7*0.14/48.0+1.12/48.0, speed=0.1, force=0.3, block=False)
    rospy.sleep(1)
    rob.translate((0, 0, 0.08), acc=0.3, vel=0.8)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.019, speed=0.5, force=10, block=True)
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)

In [16]:
from shapely.affinity import rotate
from shapely.affinity import translate
from shapely.geometry import Point
from shapely.geometry import LinearRing

def direct_pick_collision_check(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    finger_profile = [[0.01, gripper_aperture_set[Go_index]/2+0.015], [-0.01, gripper_aperture_set[Go_index]/2+0.015], [-0.01, -gripper_aperture_set[Go_index]/2-0.006], [0.01, -gripper_aperture_set[Go_index]/2-0.006]]
    finger_profile = rotate(LinearRing(finger_profile), -gripper_direction_set[Go_index], origin=Point([0,0]), use_radians=True)
    finger_profile = translate(LinearRing(finger_profile), xoff=ver_pose_Go_stone_set[Go_index][0], yoff=ver_pose_Go_stone_set[Go_index][1], zoff=0.0)
    if finger_profile.within(Point([0.05,0.69]).buffer(0.055))==True:
        return False
    else:
        return True

In [32]:
def distane_to_bowl_center(point_coordinate):
    return sqrt((point_coordinate[0]-0.05)**2+(point_coordinate[1]-0.69)**2)

stone_img_count = 0
picking_count = 0
while True:
    whether_successful = False
    whether_executed = False
    while True:
        if_continue = raw_input("continue?")
        if if_continue != '':
            break
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    go_to_home()
    rospy.sleep(1)
    hori_Go_stone_number = None
    ver_Go_stone_number = None
    Go_stone_dir_sorted = []
    hori_pose_Go_stone_set = []
    yaw_pitch_normal_set = []
    ver_pose_Go_stone_set = []
    gripper_direction_set = []
    gripper_aperture_set = []
    is_send_stone_img = 1
    print 'started', stone_img_count
    while True:
        if hori_Go_stone_number != None and hori_Go_stone_number == len(yaw_pitch_normal_set) and ver_Go_stone_number != None and ver_Go_stone_number == len(gripper_aperture_set):
            break
        rospy.sleep(0.1)   
    print hori_pose_Go_stone_set
    print yaw_pitch_normal_set
    print ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    priority_list = []
    if ver_pose_Go_stone_set == []:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            priority_list.append(('h', index_hori))
    elif hori_pose_Go_stone_set == []:
        for index_ver in range(len(ver_pose_Go_stone_set)):
            priority_list.append(('v', index_ver))
    else:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            for index_ver in range(len(ver_pose_Go_stone_set)):
                if hori_pose_Go_stone_set[index_hori][2]<=ver_pose_Go_stone_set[index_ver][2]-0.002 and ('v', index_ver) not in priority_list:
                    priority_list.append(('v', index_ver))
                v_num = 0
                for element in priority_list:
                    if element[0]=='v':
                        v_num+=1
                if (v_num == len(ver_pose_Go_stone_set) or hori_pose_Go_stone_set[index_hori][2]>ver_pose_Go_stone_set[index_ver][2]-0.002) and ('h', index_hori) not in priority_list:
                    priority_list.append(('h', index_hori))
        for index_ver in range(len(ver_pose_Go_stone_set)):
            if ('v', index_ver) not in priority_list:
                priority_list.append(('v', index_ver))
    print priority_list
    for index in priority_list:
        '''
        if index[0]=='v':
            if ver_pose_Go_stone_set[index[1]]==[None, None, None]:
                continue
            if direct_pick_collision_check(index[1])==False and ver_pose_Go_stone_set[index[1]][2]<0.075:
                direct_pick_execute(index[1])
                break
        '''
        if index[0]=='h':
            if hori_pose_Go_stone_set[index[1]]==[None, None, None]:
                continue  
            if hori_pose_Go_stone_set[index[1]][2]>0.05:
                if distane_to_bowl_center(hori_pose_Go_stone_set[index[1]])<0.02:
                    if poke_grasp_execute(index[1])==True:
                        break 
            else:
                if scoop_collision_check(index[1])==False:
                    scoop_execute(index[1]) 
                    break
    else:
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001
        if ver_pose_Go_stone_set == []:
            min_height = np.min(np.array(hori_pose_Go_stone_set)[:,2])
        elif hori_pose_Go_stone_set == []:
            min_height = np.min(np.array(ver_pose_Go_stone_set)[:,2])
        else:
            min_height = min(np.min(np.array(hori_pose_Go_stone_set)[:,2]), np.min(np.array(ver_pose_Go_stone_set)[:,2]))

        if min_height>0.058:
            rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,-0.04), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,0.04), acc=0.1, vel=0.4, wait=True) 
        elif min_height>0.053:
            rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,-0.05), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,0.05), acc=0.1, vel=0.4, wait=True) 
        elif min_height>0.045:
            rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,-0.06), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,0.06), acc=0.1, vel=0.4, wait=True) 
        else:
            if hori_pose_Go_stone_set==[]:
                rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,0,-0.075), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,0,0.075), acc=0.01, vel=0.05, wait=True)      
            else:
                servo_pub.publish(100)
                if len(hori_pose_Go_stone_set)<=2:
                    sweep_index_set = range(len(hori_pose_Go_stone_set))
                else:
                    sweep_index_set = random.sample(range(len(hori_pose_Go_stone_set)), 2)
                for hori_index in sweep_index_set:
                    alpha = atan2(0.69-hori_pose_Go_stone_set[hori_index][1], 0.05-hori_pose_Go_stone_set[hori_index][0])
                    if alpha > pi/2 and alpha<=pi:
                        rotate_angle = alpha-3*pi/2
                    else:
                        rotate_angle = alpha+pi/2
                    print alpha, rotate_angle
                    rob.movel_tool((0,0,0,0,0,-rotate_angle), acc=0.1, vel=0.2, wait=True) 
                    world2tcp = rob.get_pose()
                    world2tcp.pos.x+=0.004
                    world2tcp.pos.y-=0.002
                    tcp2fingertip=m3d.Transform()
                    tcp2fingertip.pos.x = 0
                    tcp2fingertip.pos.y = 0  
                    tcp2fingertip.pos.z = 0.354
                    world2fingertip=(world2tcp*tcp2fingertip)
                    #coordinate_x = max(hori_pose_Go_stone_set[hori_index][0], 0.008)
                    #coordinate_y = min(hori_pose_Go_stone_set[hori_index][1], 0.73)
                    rob.translate((hori_pose_Go_stone_set[hori_index][0]-world2fingertip.pos.x,hori_pose_Go_stone_set[hori_index][1]-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
                    rob.movel_tool((0.002,0,0,0,0,0), acc=0.1, vel=0.2, wait=True) 
                    rob.set_tcp((0.002, 0, 0.354, 0, 0, 0))
                    rospy.sleep(0.5)
                    rob.movel_tool((0,0,0,0,20*pi/180,0), acc=0.3, vel=0.8, wait=True) 
                    rob.set_tcp((0, 0, 0, 0, 0, 0))
                    rospy.sleep(0.5)
                    rob.translate((0, 0, -0.05), acc=0.1, vel=0.3, wait=True)
                    rob.translate((0, 0, -0.05), acc=0.001, vel=0.02, wait=False)
                    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                    ini_wrench = wrench.wrench.force.z
                    torque_memory=[]
                    while True:
                        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                        torque_memory.append(wrench.wrench.force.z)
                        if len(torque_memory)<9:
                            if wrench.wrench.force.z<ini_wrench-5:
                                rob.stopl()
                                break
                        else:
                            if wrench.wrench.force.z<max(torque_memory[0:9])-5:
                                rob.stopl()
                                break 
                    rob.translate((0.035*cos(alpha), 0.035*sin(alpha), 0), acc=0.1, vel=0.1, wait=True)
                    #hardtip()
                    rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)
                    rob.movej((radians(99.10),radians(-91.67), radians(-107.60), radians(-70.72), radians(89.13), radians(-80.76)), 0.3, 0.7)

continue?1
started 0
dsag
0
[[0.04422073627167999, 0.6852887275198208, 0.06940791315529399], [0.06600877505778809, 0.6657713468619338, 0.0639819261620076], [0.024436523054328606, 0.687308819532448, 0.06201560734939149], [0.07433409012197768, 0.6841080799314799, 0.05930722592969556]]
[[-2.7801517626962404, 0.4101273405414903, (-0.23705510698727605, -0.2126411923860918, -0.646381745673756)], [-2.995108475196232, 0.6365082157879507, (0.0589453922450623, 0.26163246153242165, -0.9633659199938318)], [-2.9441970937399122, 0.7399748839758711, (0.19752320856784653, -0.03401979717762451, -0.8794045369024241)], [2.6158241196006022, 0.8270407424960367, (0.2171914816397498, 0.20977497196455022, -0.7658836164754317)]]
[[0.03136554562303001, 0.6381136458517002, 0.06266302648969196], [0.012911734917054177, 0.6927875261283764, 0.07140443984064881], [-0.0006506345299945888, 0.6857742171376969, 0.07699767848615965], [0.0026091482848188893, 0.6916502679500932, 0.06811837379169156]] [0.0718731024887751, -0

KeyboardInterrupt: 

In [18]:
((0.075-0.07462983631818886)+(0.05-0.04978197174341037)+(0.025-0.02363016791729483)+(0.025-0.024555630670864904))/4.0

0.0006005983375602609

In [19]:
((0.675-0.6767408309761674)+(0.675-0.6758921484067598)+(0.7-0.6993785066287045)+(0.65-0.6487997658859069))/4.0

-0.00020281297438465296

In [22]:
Robotiq.goto(robotiq_client, pos=0.021, speed=0.000001, force=1, block=True)
#servo_pub.publish(180)
#softtip()

In [30]:
Robotiq.goto(robotiq_client, pos=0.8, speed=0.000001, force=1, block=True)

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 291, in run
    data = self._get_data()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 338, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out



In [81]:
131, 0.06473477762172797], [0.05002869920583822, 0.668622249134879, 0.06071871600676282], [0.06440163964727431, 
                    np.save('collecting_data/'+dir_
Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)

rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/5,0),acc=0.1, vel=0.2)
rob.movel_tool((0,0,0,0,-pi/5,0),acc=0.1, vel=0.2)
rob.set_tcp((0.002, 0, 0.356, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/5,0),acc=0.1, vel=0.2)
rob.movel_tool((0,0,0,0,-pi/5,0),acc=0.1, vel=0.2)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/5,0),acc=0.1, vel=0.2)
rob.movel_tool((0,0,0,0,-pi/5,0),acc=0.1, vel=0.2)

KeyboardInterrupt: 

In [25]:
go_to_safe_place()
softtip()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)

<Transform:
<Orientation: 
array([[-0.01305109,  0.99989524,  0.00625989],
       [ 0.99991361,  0.01304105,  0.00164169],
       [ 0.00155988,  0.00628078, -0.99997906]])>
<Vector: (0.04293, 0.85097, 0.45825)>
>

In [9]:
go_to_safe_place()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
rob.translate_tool((0.0065,0,0),acc=0.1, vel=0.2)

<Transform:
<Orientation: 
array([[-0.01308933,  0.99989511,  0.00619996],
       [ 0.99991303,  0.01307907,  0.00169334],
       [ 0.00161207,  0.00622159, -0.99997935]])>
<Vector: (0.04282, 0.80750, 0.45825)>
>

In [23]:
go_to_safe_place()
rob.set_tcp((0.002, 0, 0.356, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,0,0*pi/180), acc=0.1, vel=0.2, wait=True)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
#rob.movel_tool((0,0,0,0,-20*pi/180,0), acc=0.1, vel=0.2, wait=True)
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
#go_to_safe_place()
servo_pub.publish(0)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = 0.002
tcp2fingertip.pos.y = 0  
tcp2fingertip.pos.z = 0.356
world2tcp = rob.get_pose()
world2tcp.pos.x+=0.004
world2tcp.pos.y+=0.001
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.1, vel=0.4, wait=True)
servo_pub.publish(100)
rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)


<Transform:
<Orientation: 
array([[-0.01301363,  0.99989673,  0.00609763],
       [ 0.99991311,  0.01300053,  0.00218286],
       [ 0.00210337,  0.00612551, -0.99997903]])>
<Vector: (0.04382, 0.84645, 0.39055)>
>

In [21]:
rob.set_tcp((0.002, 0, 0.356, 0, 0, 0))
rospy.sleep(0.5)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
rob.movel_tool((0,0,0,0,-pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.movel_tool((0,0,0,0,pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [26]:
go_to_safe_place()
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rob.translate((0.05-pos_x+0.02,0.69-pos_y-0.02,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0,-0.06), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0.04,0), acc=0.1, vel=0.4, wait=True)
rob.translate((-0.04,0,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,-0.04,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0.04,0,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0,0.06), acc=0.1, vel=0.4, wait=True)  

<Transform:
<Orientation: 
array([[-0.01328703,  0.99989138,  0.00637776],
       [ 0.99991118,  0.0132801 ,  0.00112777],
       [ 0.00104295,  0.00639218, -0.99997903]])>
<Vector: (0.06282, 0.67096, 0.45798)>
>

In [37]:
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001

        rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,-0.075), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,0.075), acc=0.01, vel=0.05, wait=True)     


<Transform:
<Orientation: 
array([[-0.01329421,  0.99989335,  0.00604565],
       [ 0.99990707,  0.01327565,  0.00310065],
       [ 0.00302006,  0.00608631, -0.99997692]])>
<Vector: (0.07290, 0.66112, 0.45890)>
>

In [20]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 291, in run
    data = self._get_data()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 338, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out

[WARN] [1605933605.661097]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [ ]:
go_to_safe_place()
Robotiq.goto(robotiq_client, pos=5*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
servo_pub.publish(0)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = -0.021 # need change
tcp2fingertip.pos.z = 0.342 # need change
world2tcp = rob.get_pose()
world2tcp.pos.x+=0
world2tcp.pos.y-=0
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)

In [5]:
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    tcp2cam = np.matmul(tcpTee, eeTcam.get_matrix())
    print tcp2cam

[[ 0.02153275 -0.9995875  -0.01900464  0.1234057 ]
 [ 0.99884698  0.02232484 -0.04250071 -0.0074811 ]
 [ 0.04290745 -0.01806757  0.99891567  0.126173  ]
 [ 0.          0.          0.          1.        ]]


In [62]:
softtip()

In [22]:
print np.load('/home/terry/catkin_ws/src/scoop/src/collecting_data/Mon_Nov_23_19_36_07_2020/executed_action_list.npy')

[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]]


In [33]:
go_to_safe_place()

In [24]:
softtip()

In [35]:
    servo_pub.publish(180)

In [4]:
rob.set_tcp((0, 0, 0.35947, 0, 0, 0))
rospy.sleep(1)
print rob.get_pose()
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(1)

<Transform:
<Orientation: 
array([[-0.01308581,  0.99989541,  0.00615819],
       [ 0.99991336,  0.01307677,  0.0015056 ],
       [ 0.00142491,  0.00617736, -0.9999799 ]])>
<Vector: (0.04869, 0.39967, -0.01995)>
>


In [3]:
import numpy as np
print np.dot(np.array([1,2,3]), np.array([2,-1,1]))
print np.linalg.norm(np.array([1,2,3]))
print np.linalg.norm(np.array([2,-1,1]))

3
3.7416573867739413
2.449489742783178


In [34]:
    Robotiq.goto(robotiq_client, pos=0.8, speed=0.1, force=10, block=True)
    servo_pub.publish(85) # need change

In [29]:
        current_alpha = round((0.021-1.12/48)/(0.14/48))   # need change
        angle_step = 0.2
        l0 = 0.0125
        l1 = 0.1
        l2r = 0.006
        flength_r = 0.126
        for alpha in np.arange(current_alpha,1.5,angle_step):
            next_alpha = alpha+0.2
            alpha = alpha*pi/180
            next_alpha = next_alpha*pi/180
            translate_distance = (np.array([l0+l1*sin(next_alpha)-l2r, -l1*cos(next_alpha)-flength_r])-np.array([l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r])).tolist()
            Robotiq.goto(robotiq_client, pos=(next_alpha*180/pi)*0.14/48.0+1.12/48.0, speed=0.4, force=10, block=False)
            rob.translate_tool((-translate_distance[0], 0, -translate_distance[1]), acc=0.1, vel=0.4, wait=True)       
        servo_pub.publish(115)
        rob.set_tcp((0.019, 0, 0.342, 0, 0, 0)) # need change!!!
        rospy.sleep(0.5)
        minus_x = -rob.get_pose().get_matrix()[0:3,0]
        minus_x_proj = [minus_x[0], minus_x[1], 0]
        minus_z = -rob.get_pose().get_matrix()[0:3,2]
        rotate_angle = -(60*pi/180-acos(np.dot(minus_x_proj, minus_z)/(np.linalg.norm(minus_x_proj)*np.linalg.norm(minus_z))))
        rob.movel_tool((0,0,0,0,rotate_angle,0), acc=0.3, vel=0.2, wait=True)
        rob.set_tcp((0, 0, 0, 0, 0, 0))
        rospy.sleep(0.5)
        current_fingerlength_int=115   #need change
        angle_step=0.2
        for angle in np.arange(1.5,-4,-angle_step):
            translate_dir_dis, strench_distance = pos_flength_adjust_right_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.015, 0.006, 0.126)
            old_fingerlength_int=current_fingerlength_int
            current_fingerlength_int=round(min(current_fingerlength_int+strench_distance*(180.0/0.03),180))
            servo_pub.publish(int(current_fingerlength_int))
            Robotiq.goto(robotiq_client, pos=angle*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        Robotiq.goto(robotiq_client, pos=-4*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
        rospy.sleep(0.5)

KeyboardInterrupt: 

In [56]:
    Robotiq.goto(robotiq_client, pos=0.021, speed=0.1, force=10, block=True)


In [55]:
servo_pub.publish(150)

In [36]:
    rob.translate_tool((0, 0, 0.0285), acc=0.01, vel=0.05, wait=False) #2.85 a003 v005  $0.285
    
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.force.z
    time0 = time.time()
    IsStack = False
    while True:
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        print wrench.wrench.force.z
        if time.time()-time0>3.7:
            break

81.2699966431
81.1699981689
81.2900009155
80.8499984741
80.9899978638
80.5800018311
80.5
81.1999969482
81.1200027466
80.9700012207
81.0699996948
80.75
80.5199966431
80.4499969482
80.3199996948
80.5699996948
81.2600021362
80.9000015259
80.5999984741
80.4400024414
80.6999969482
80.5899963379
80.6699981689
80.7900009155
80.7699966431
80.4899978638
80.4899978638
80.5400009155
81.3700027466
81.1699981689
80.7300033569
81.25
81.2699966431
81.1100006104
81.2200012207
81.1900024414
80.4800033569
80.6200027466
80.0899963379
80.0400009155
80.3799972534
80.6200027466
80.2300033569
80.1999969482
80.2399978638
80.5899963379
80.4499969482
80.7099990845
80.6200027466
80.4499969482
80.4300003052
80.2699966431
80.3300018311
80.4800033569
80.5699996948
80.3300018311
80.2399978638
80.4000015259
80.2200012207
80.1200027466
80.8000030518
80.6600036621
80.7600021362
80.6100006104
80.4000015259
80.3799972534
80.1100006104
80.0500030518
79.6399993896
79.8799972534
78.9800033569
77.3199996948
76.9499969482
78.

In [14]:
import pybullet as p
import time
import pybullet_data
time0 = time.time()
physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
planeId = p.loadURDF("plane.urdf")
cubeStartPos = [0,0,0]
cubeStartOrientation = p.getQuaternionFromEuler([0,0,0])
boxId = p.loadURDF("r2d2.urdf",cubeStartPos, cubeStartOrientation)
plts = p.getClosestPoints(bodyA=planeId, bodyB=boxId, distance=0)
if plts==():
    print 'No collision'
else:
    print 'collision'
p.disconnect()
print time.time()-time0

0.0142788887024
collision
